In [31]:
import os, sys
import re
from dotenv import load_dotenv
import time
import pandas as pd
from openai import AzureOpenAI, AsyncAzureOpenAI
import tiktoken
import asyncio

In [2]:
load_dotenv('.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', None)

In [3]:
token_encoder = tiktoken.get_encoding('cl100k_base')
def count_tokens(txt):
    return len(token_encoder.encode(txt))

In [156]:
dbt_skills_ref = pd.read_csv(
    os.path.join('..', 'data', 'dbt_skills_ref.csv'),
    index_col='skill_id',
)
dbt_skills_ref

,skill_name,module_focus,module_name,category_name,skill_desc,regex_patterns
skill_id,,,,,,
M1,Wise Mind,Acceptance,Mindfulness,NaN,"- The ""Wise Mind"" skill in the Mindfulness mod...",.*\b[Ww]ise [Mm]ind\b.*
M2,Observing,Acceptance,Mindfulness,"Mindfulness ""What"" Skills","- The ""Observing"" skill in the Mindfulness mod...",NaN
M3,Describing,Acceptance,Mindfulness,"Mindfulness ""What"" Skills","- The ""Describing"" skill in the Mindfulness mo...",NaN
M4,Participating,Acceptance,Mindfulness,"Mindfulness ""What"" Skills","- The ""Participating"" skill in the Mindfulness...",NaN
M5,Nonjudgmentally,Acceptance,Mindfulness,"Mindfulness ""How"" Skills","- The ""Nonjudgmentally"" skill in the Mindfulne...",NaN
M6,One-Mindfully,Acceptance,Mindfulness,"Mindfulness ""How"" Skills","- The ""One-Mindfully"" skill in the Mindfulness...",NaN
M7,Effectively,Acceptance,Mindfulness,"Mindfulness ""How"" Skills","- The ""Effectively"" skill in the Mindfulness m...",NaN
M8,Mindfulness Practice: A Spiritual Perspective,Acceptance,Mindfulness,Other Perspectives on Mindfulness,"- The ""Mindfulness Practice: A Spiritual Persp...",NaN
M9,Balancing Doing Mind and Being Mind,Acceptance,Mindfulness,Other Perspectives on Mindfulness,"- The ""Balancing Doing Mind and Being Mind"" sk...",NaN


In [163]:
dbt_skills_ref.loc['DT10']['regex_patterns']

'.*(\\b[Hh]alf-[Ss]mil(e|ing)\\b)|(\\b[Ww]illing [Hh]ands\\b).*'

In [167]:
dbt_skills_ref.regex_patterns[pd.notna(dbt_skills_ref.regex_patterns)]

skill_id
M1                                .*\b[Ww]ise [Mm]ind\b.*
IE2                                      .*\bDEAR MAN\b.*
IE3                                          .*\bGIVE\b.*
IE4                                          .*\bFAST\b.*
IE7                   .*\b[Mm]indfulness of [Oo]thers\b.*
ER2                   .*\b[Cc]heck(ing)? the [Ff]acts\b.*
ER3                         .*\b[Oo]pposite [Aa]ction\b.*
ER5     .*\b[Aa]ccumulat(e|ing) [Pp]ositive [Ee]motion...
ER6                     .*\b[Bb]uild(ing)? [Mm]astery\b.*
ER7                        .*\b[Cc]op(e|ing) [Aa]head\b.*
ER8                                        .*\bPLEASE\b.*
ER9      .*\b[Mm]indfulness of [Cc]urrent [Ee]motions\b.*
DT1                                          .*\bSTOP\b.*
DT2                           .*\b[Pp]ros and [Cc]ons\b.*
DT3                                      .*\bTIP{1,2}\b.*
DT4                                       .*\bACCEPTS\b.*
DT5                           .*\b[Ss]elf-[Ss]oothing\b.*
DT6  

In [5]:
dbt_skills_ref.skill_desc[0]

'- The "Wise Mind" skill in the Mindfulness module of DBT refers to a state of mind where an individual integrates both rational thought (Reasonable Mind) and emotional experience (Emotion Mind) to make balanced and effective decisions.\n- It is considered a core concept in DBT, representing an inner knowledge and intuition that everyone possesses, which can guide them to a sense of truth and balanced action.\n- In practice, this skill involves recognizing when one is operating from either the Reasonable Mind or the Emotion Mind and then working towards achieving a synthesis of the two, which is the Wise Mind.\n- During a session, it might be identified by the therapist asking the client to reflect on a situation and consider what their emotions, reason, and intuition are telling them, aiming to find a middle path that incorporates all aspects.'

In [6]:
convos = pd.read_pickle(os.path.join('..', 'convo_generation', 'output', '20240306_convos_full.pickle'))

In [7]:
skill_extract_test = convos['bc97e560-2f17-40c6-bbbc-cf1455e5536b']['result']['eval_messages']
print(skill_extract_test)

Therapist: Hello and welcome! I'm glad you're here to explore Dialectical Behavior Therapy (DBT) skills. Is there a particular DBT skill or technique you're interested in practicing today, or would you like me to suggest one based on what you're experiencing currently?

Client: My grandmother passed away two days ago and everyone in my family has been busy grieving so I've been picking up all the preparations for her wake. I haven't had a chance to stop but if I don't take care of everything, I don't think anyone else will.

Therapist: I'm so sorry to hear about your loss. It sounds like you’re carrying a heavy burden during this tough time. Grief can be overwhelming, and it's important to take care of yourself too. How are you currently feeling in this moment as you handle these responsibilities?

Client: Thank you for your kind words. I'm feeling incredibly overwhelmed and exhausted. It's like I'm running on autopilot, trying to keep it all together for my family's sake, but inside I

In [26]:
count_tokens(skill_extract_test)

595

In [9]:
client = AzureOpenAI(
    api_key = OPENAI_API_KEY,
    api_version = "2023-05-15",
    azure_endpoint = "https://dbt-openai-usea2-assistants.openai.azure.com/"
)

In [10]:
client_async = AsyncAzureOpenAI(
    api_key = OPENAI_API_KEY,
    api_version = "2023-05-15",
    azure_endpoint = "https://dbt-openai-usea2-assistants.openai.azure.com/"
)

In [11]:
skill_extract_base_prompt = '''
You are tasked with evaluating whether a therapist discussed the following DBT skill with their client during a session:
<SKILL_DESC>

Carefully analyze the provided conversation log between a therapist and their client, then assign a numerical rating based on the following criteria.
Assign a "1" for "present" if the therapist:
- Explicitly refers to this skill by name.
- Introduces, suggests, defines, or explains this skill to the client.
- Teaches this skill to the client.
- Guides the client in practicing this skill.
- Assists the client in applying this skill to their life.
Assign a "0" for "not present" if the therapist:
- Does not refer to this skill by name.
- Does not introduce, suggest, define, or explain this skill to the client.
- Does not teach this skill to the client.
- Does not guide the client in practicing this skill.
- Does not assist the client in applying this skill to their life.

Instructions:
- Please provide your rating as a standalone numerical value, with no additional comments or justification.
- Valid responses are limited to: [1, 0]
'''.strip()

In [12]:
skill_extract_base_prompt2 = '''
Determine if a therapist discussed the following Dialectical Behavior Therapy (DBT) skill during a session:
<SKILL_DESC>

Carefully review the provided transcript between the therapist and client. Rate whether the therapist discussed the DBT skill using the following criteria:

Rate "1" if the therapist did any of the following:
- Mentioned the skill by name.
- Defined, described, explained, or taught the skill to the client.
- Led the client through guided practice of the skill.
- Helped apply the skill to the client's life.

Rate "0" if none of these actions took place.

Provide your rating as a single number, either "1" or "0", with no further explanation.
'''.strip()

In [249]:
with open(os.path.join('..', 'data', 'skill_presence_base_prompt.txt'), 'wt') as f:
    f.write(skill_extract_base_prompt2)

In [19]:
t_start = time.perf_counter()

for row in dbt_skills_ref.itertuples():
    row_sys_prompt = skill_extract_base_prompt2.replace("<SKILL_DESC>", row.skill_desc)
    response = client.chat.completions.create(
        # model="gpt4-1106",
        model="gpt-35-turbo",
        messages=[
            {"role": "system", "content": row_sys_prompt},
            {"role": "user", "content": skill_extract_test}
        ],
        n=5,
        temperature=0.
    )
    ratings = [choice.message.content for choice in response.choices]
    print(f'{row.skill_name}: {ratings}')

t_stop = time.perf_counter()
elapsed_time = t_stop - t_start
print(elapsed_time)

Wise Mind: ['0', '0', '0', '0', '0']
Observing: ['0', '0', '0', '0', '0']
Describing: ['0', '0', '0', '0', '0']
Participating: ['0', '0', '0', '0', '0']
Nonjudgmentally: ['0', '0', '0', '0', '0']
One-Mindfully: ['0', '0', '0', '0', '0']
Effectively: ['0', '0', '0', '0', '0']
Mindfulness Practice: A Spiritual Perspective: ['0', '0', '0', '0', '0']
Balancing Doing Mind and Being Mind: ['0', '0', '0', '0', '0']
Clarifying Priorities: ['0', '0', '0', '0', '0']
DEAR MAN: ['0', '0', '0', '0', '0']
GIVE: ['0', '0', '0', '0', '0']
FAST: ['0', '0', '0', '0', '0']
Whether and How Intensely to Ask or Say No: ['0', '0', '0', '0', '0']
Finding Potential Friends: ['0', '0', '0', '0', '0']
Mindfulness of Others: ['0', '0', '0', '0', '0']
How to End Relationships: ['0', '0', '0', '0', '0']
Dialectics: ['0', '0', '0', '0', '0']
Validation: ['0', '0', '0', '0', '0']
Behavior Change Strategies: ['0', '0', '0', '0', '0']
Understanding and Naming Emotions: ['0', '0', '0', '0', '0']
Checking the Facts: ['0'

In [ ]:
async def get_skill_presence_response(client, sys_prompt, eval_msgs, n):
    return await client.chat.completions.create(
        model='gpt4-1106',
        # model="gpt-35-turbo",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": eval_msgs},
        ],
        n=n,
        temperature=0.,
    )

async def get_skill_presences(client, sys_prompt_base, eval_msgs, n):
    skill_names = list(dbt_skills_ref.skill_name)[:]
    skill_descs = list(dbt_skills_ref.skill_desc)[:]
    sys_prompts = [sys_prompt_base.replace("<SKILL_DESC>", desc) for desc in skill_descs]
    tasks = [get_skill_presence_response(client, sys_prompt, eval_msgs, n) for sys_prompt in sys_prompts]
    responses = await asyncio.gather(*tasks)
    for i, skill_name in enumerate(skill_names):
        ratings = [choice.message.content for choice in responses[i].choices]
        print(f'{skill_name}: {ratings}')

t_start = time.perf_counter()
await get_skill_presences(client_async, skill_extract_base_prompt2, skill_extract_test, 5)
t_stop = time.perf_counter()
print(t_stop - t_start)

In [19]:
async def get_skill_presence_response(client, sys_prompt, eval_msgs, n):
    response = await client.chat.completions.create(
        model='gpt4-1106',
        # model="gpt-35-turbo",
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": eval_msgs},
        ],
        n=n,
        temperature=0.,
    )
    return [choice.message.content for choice in response.choices]

async def get_skill_presences(client, sys_prompt_base, eval_msgs, n):
    skill_names = list(dbt_skills_ref.skill_name)[:]
    skill_descs = list(dbt_skills_ref.skill_desc)[:]
    sys_prompts = [sys_prompt_base.replace("<SKILL_DESC>", desc) for desc in skill_descs]
    skill_presence_dict = dict.fromkeys(skill_names)
    for i, skill_name in enumerate(skill_names):
        skill_presence_dict[skill_name] = await asyncio.create_task(get_skill_presence_response(client, sys_prompts[i], eval_msgs, n))
        await asyncio.sleep(5)
    print(skill_presence_dict)

    # tasks = [get_skill_presence_response(client, sys_prompt, eval_msgs, n) for sys_prompt in sys_prompts]
    # responses = await asyncio.gather(*tasks)
    # for i, skill_name in enumerate(skill_names):
    #     ratings = [choice.message.content for choice in responses[i].choices]
    #     print(f'{skill_name}: {ratings}')

t_start = time.perf_counter()
await get_skill_presences(client_async, skill_extract_base_prompt2, skill_extract_test, 5)
t_stop = time.perf_counter()
print(t_stop - t_start)

{'Wise Mind': ['0', '0', '0', '0', '0'], 'Observing': ['0', '0', '0', '0', '0'], 'Describing': ['0', '0', '0', '0', '0'], 'Participating': ['0', '0', '0', '0', '0'], 'Nonjudgmentally': ['0', '0', '0', '0', '0'], 'One-Mindfully': ['0', '0', '0', '0', '0'], 'Effectively': ['0', '0', '0', '0', '0'], 'Mindfulness Practice: A Spiritual Perspective': ['0', '0', '0', '0', '0'], 'Balancing Doing Mind and Being Mind': ['0', '0', '0', '0', '0'], 'Clarifying Priorities': ['0', '0', '0', '0', '0'], 'DEAR MAN': ['0', '0', '0', '0', '0'], 'GIVE': ['0', '0', '0', '0', '0'], 'FAST': ['0', '0', '0', '0', '0'], 'Whether and How Intensely to Ask or Say No': ['0', '0', '0', '0', '0'], 'Finding Potential Friends': ['0', '0', '0', '0', '0'], 'Mindfulness of Others': ['0', '0', '0', '0', '0'], 'How to End Relationships': ['0', '0', '0', '0', '0'], 'Dialectics': ['0', '0', '0', '0', '0'], 'Validation': ['0', '0', '0', '0', '0'], 'Behavior Change Strategies': ['0', '0', '0', '0', '0'], 'Understanding and Namin

In [18]:
async def get_skill_presence_response(client, semaphore, sys_prompt, eval_msgs):
    async with semaphore:
        return await client.chat.completions.create(
            model='gpt4-1106',
            # model="gpt-35-turbo",
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": eval_msgs},
            ],
            temperature=0.,
        )

async def get_skill_presences(
        client:AsyncAzureOpenAI,
        max_parallel_calls:int,
        sys_prompt_base:str,
        eval_msgs:str,
        n_replications:int|None = None):
    assert max_parallel_calls > 0
    if n_replications:
        assert n_replications > 0
    skill_names = list(dbt_skills_ref.skill_name)[:5]
    skill_descs = list(dbt_skills_ref.skill_desc)[:5]
    sys_prompts = [sys_prompt_base.replace("<SKILL_DESC>", desc) for desc in skill_descs]
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    tasks = [get_skill_presence_response(client, semaphore, sys_prompt, eval_msgs) for sys_prompt in sys_prompts]
    responses = await asyncio.gather(*tasks)
    for i, skill_name in enumerate(skill_names):
        ratings = [choice.message.content for choice in responses[i].choices]
        print(f'{skill_name}: {ratings}')

t_start = time.perf_counter()
await get_skill_presences(client_async, 5, skill_extract_base_prompt2, skill_extract_test, 3)
t_stop = time.perf_counter()
print(t_stop - t_start)

Wise Mind: ['0']
Observing: ['0']
Describing: ['0']
Participating: ['0']
Nonjudgmentally: ['0']
3.857143000001088


In [19]:
skill_names = list(dbt_skills_ref.skill_name)[:5]
skill_names

['Wise Mind', 'Observing', 'Describing', 'Participating', 'Nonjudgmentally']

In [20]:
[skill_name for skill_name in skill_names for i in range(3)]

['Wise Mind',
 'Wise Mind',
 'Wise Mind',
 'Observing',
 'Observing',
 'Observing',
 'Describing',
 'Describing',
 'Describing',
 'Participating',
 'Participating',
 'Participating',
 'Nonjudgmentally',
 'Nonjudgmentally',
 'Nonjudgmentally']

In [41]:
eval_rewrite_prompt = '''
You will be given a GPT system prompt used for evaluating whether a therapist discussed a specific Dialectical Behavior Therapy (DBT) skill with their client during a session.
Rewrite and reorganize this system prompt so that it is:
- More effective in guiding GPT behavior to match the intended purpose
- Clearly worded
- More concise
'''.strip()

In [42]:
response = client.chat.completions.create(
    model="gpt4-1106",
    messages=[
        {"role": "system", "content": eval_rewrite_prompt},
        {"role": "user", "content": skill_extract_base_prompt}
    ],
    n=5,
)
ratings = [choice.message.content for choice in response.choices]
for choice in response.choices:
    print(choice.message.content, end='\n\n'+'-'*30+'\n\n')

Determine if a therapist discussed a specific Dialectical Behavior Therapy (DBT) skill during a session:

DBT Skill: <SKILL_DESC>

Review the transcript between the therapist and client. Rate whether the therapist discussed the DBT skill using the following criteria:

Rate "1" for "present" if the therapist:
- Named the skill.
- Explained or defined the skill.
- Instructed or educated the client about the skill.
- Supported the client in practicing the skill.
- Aided the client in applying the skill to situations.

Rate "0" for "not present" if:
- The skill was not mentioned.
- The skill was not explained or defined.
- The skill was not taught or covered.
- The client was not supported in practicing the skill.
- The client received no help in applying the skill.

Provide your response as a single number, either "1" or "0". No justification or additional comments are needed.

------------------------------

Evaluate whether a DBT skill was discussed by a therapist as described by <SKILL

In [117]:
regex_prompt = r'''
Act as an expert in natural language processing who is familiar with Perl-style regular expressions.
You will be given a description of a Dialectical Behavior Therapy (DBT) skill.
Given this DBT skill description, come up with a few regular expressions which would reliably allow you to determine whether this DBT skill was discussed during a therapy session transcript.

Example Input:
- The "TIP" skill in the Distress Tolerance module of DBT stands for Temperature, Intense exercise, Paced breathing, and Paired muscle relaxation.
- It is designed to rapidly alter body chemistry to reduce intense emotional arousal by engaging the parasympathetic nervous system and calming the sympathetic nervous system.
- In a session, this skill might be identified by a therapist instructing a client to hold their breath and then plunge their face into cold water (Temperature), engage in a brief burst of intense physical activity (Intense exercise), practice slow, deep breaths (Paced breathing), or tense and then relax their muscles in sequence (Paired muscle relaxation).

Example Responses:
```
.*\bTIP{1,2}\b.*
```

```
.*\b[Tt]emperature\b.*\b[Ii]ntense [Ee]xercise\b.*\b[Pp]aced [Bb]reath(ing|s)?\b.*\b[Pp]aired [Mm]uscle [Rr]elaxation\b.*
```
'''.strip()

In [120]:
regex_skill_desc = '''
- The "FAST" skill in the Interpersonal Effectiveness module of DBT focuses on maintaining self-respect during interactions with others.
- FAST stands for (be) Fair, (no) Apologies for being alive or for having an opinion, Stick to values, and (be) Truthful.
- This skill is used to help individuals assert their needs and opinions without compromising their own values and integrity.
- In a session, this skill would be identified by the therapist guiding the client through one or more of the steps in "FAST."
'''.strip()

In [121]:
response = client.chat.completions.create(
    model="gpt4-1106",
    messages=[
        {"role": "system", "content": regex_prompt},
        {"role": "user", "content": regex_skill_desc}
    ],
)
print(response.choices[0].message.content)

Based on the description provided for the "FAST" skill in DBT, you would want to create regular expressions that capture any mention or discussion of these concepts within a therapy session transcript. Here are a few regular expressions that could help identify whether the "FAST" skill was discussed:

1. To match the acronym "FAST" with variations in case:
```
.*\bFAST\b.*
.*\bF(?i)ast\b.*
```

2. To match each component of the "FAST" skill with some allowance for variations in phrasing:
```
.*\b(be )?F(?i)air\b.*
.*\b(no )?[Aa]pologies\b.*(alive|opinion)\b.*
.*\b[Ss]tick\b.*\b[Tt]o\b.*\b[Vv]alues\b.*
.*\b(be )?[Tt]ruthful\b.*
```

3. A comprehensive regular expression to look for any mention of all four components of FAST in no specific order:
```
.*\b(be )?F(?i)air\b.*|(.*\b(no )?[Aa]pologies\b.*(alive|opinion)\b.*)|(.*\b[Ss]tick\b.*\b[Tt]o\b.*\b[Vv]alues\b.*)|(.*\b(be )?[Tt]ruthful\b.*)
```

4. A regular expression to search for any mention of a therapist guiding a client which migh

In [33]:
print(skill_extract_test)

Therapist: Hello and welcome! I'm glad you're here to explore Dialectical Behavior Therapy (DBT) skills. Is there a particular DBT skill or technique you're interested in practicing today, or would you like me to suggest one based on what you're experiencing currently?

Client: My grandmother passed away two days ago and everyone in my family has been busy grieving so I've been picking up all the preparations for her wake. I haven't had a chance to stop but if I don't take care of everything, I don't think anyone else will.

Therapist: I'm so sorry to hear about your loss. It sounds like you’re carrying a heavy burden during this tough time. Grief can be overwhelming, and it's important to take care of yourself too. How are you currently feeling in this moment as you handle these responsibilities?

Client: Thank you for your kind words. I'm feeling incredibly overwhelmed and exhausted. It's like I'm running on autopilot, trying to keep it all together for my family's sake, but inside I

In [180]:
pattern_str = r'\bFAST\b'
pattern = re.compile(pattern_str)
match = re.search(pattern, regex_skill_desc)
match

<re.Match object; span=(7, 11), match='FAST'>

In [182]:
pattern.groups

0

In [179]:
regex_skill_desc

'- The "FAST" skill in the Interpersonal Effectiveness module of DBT focuses on maintaining self-respect during interactions with others.\n- FAST stands for (be) Fair, (no) Apologies for being alive or for having an opinion, Stick to values, and (be) Truthful.\n- This skill is used to help individuals assert their needs and opinions without compromising their own values and integrity.\n- In a session, this skill would be identified by the therapist guiding the client through one or more of the steps in "FAST."'

In [142]:
match.groups()

(None, None, None, 'ies', None, ' to values', None, None)

In [135]:
pattern_str = r'.*(\bbe\b)?\b[Ff]air\b.*'
test_str = 'be fair to me'
pattern = re.compile(pattern_str, re.S)
match = re.match(pattern, test_str)
match

<re.Match object; span=(0, 13), match='be fair to me'>

In [138]:
match.groups()

(None,)

In [237]:
SKILL_REGEX_DICT = {
    'M1':   {'pattern_str': r'\b[Ww]ise [Mm]ind\b'},
    'IE2':  {'pattern_str': r'\bDEAR MAN\b'},
    'IE3':  {'pattern_str': r'\bGIVE\b'},
    'IE4':  {'pattern_str': r'\bFAST\b'},
    'IE7':  {'pattern_str': r'\b[Mm]indfulness of [Oo]thers\b'},
    'ER2':  {'pattern_str': r'\b[Cc]heck(ing)? the [Ff]acts\b'},
    'ER3':  {'pattern_str': r'\b[Oo]pposite [Aa]ction\b'},
    'ER5':  {'pattern_str': r'\b[Aa]ccumulat(e|ing) [Pp]ositive [Ee]motions\b'},
    'ER6':  {'pattern_str': r'\b[Bb]uild(ing)? [Mm]astery\b'},
    'ER7':  {'pattern_str': r'\b[Cc]op(e|ing) [Aa]head\b'},
    'ER8':  {'pattern_str': r'\bPLEASE\b'},
    'ER9':  {'pattern_str': r'\b[Mm]indfulness of [Cc]urrent [Ee]motions\b'},
    'DT1':  {'pattern_str': r'\bSTOP\b'},
    'DT2':  {'pattern_str': r'\b[Pp]ros and [Cc]ons\b'},
    'DT3':  {'pattern_str': r'\bTIP{1,2}\b'},
    'DT4':  {'pattern_str': r'\bACCEPTS\b'},
    'DT5':  {'pattern_str': r'\b[Ss]elf-[Ss]oothing\b'},
    'DT6':  {'pattern_str': r'\bIMPROVE\b'},
    'DT7':  {'pattern_str': r'\b[Rr]adical [Aa]cceptance\b'},
    'DT8':  {'pattern_str': r'\b[Tt]urn(ing)? the [Mm]ind\b'},
    'DT10': {'pattern_str': r'(\b[Hh]alf-[Ss]mil(e|ing)\b)|(\b[Ww]illing [Hh]ands\b)'},
    'DT11': {'pattern_str': r'\b[Mm]indfulness of [Cc]urrent [Tt]houghts\b'},
    'DT12': {'pattern_str': r'\b[Dd]ialectical [Aa]bstinence\b'},
    'DT13': {'pattern_str': r'\b[Cc]lear [Mm]ind\b'},
    'DT14': {'pattern_str': r'\b[Cc]ommunity [Rr]einforcement\b'},
    'DT15': {'pattern_str': r'(\b[Bb]urning [Bb]ridges\b)|(\b[Bb]uild(ing)? [Nn]ew ([Oo]nes)|([Bb]ridges)\b)'},
    'DT16': {'pattern_str': r'(\b[Aa]lternate [Rr]ebellion\b)|(\b[Aa]daptive [Dd]enial\b)'},
}

In [190]:
for value in skill_regex_dict.values():
    value['pattern'] = re.compile(value['pattern_str'])

In [192]:
for key, value in skill_regex_dict.items():
    if re.search(value['pattern'], skill_extract_test):
        print(key)

DT3


In [208]:
DBT_SKILLS_REF = pd.read_csv(
    os.path.join('..', 'data', 'dbt_skills_ref.csv'),
    index_col='skill_id',
)
DBT_SKILLS_REF

,skill_name,module_focus,module_name,category_name,skill_desc,use_regex
skill_id,,,,,,
M1,Wise Mind,Acceptance,Mindfulness,NaN,"- The ""Wise Mind"" skill in the Mindfulness mod...",True
M2,Observing,Acceptance,Mindfulness,"Mindfulness ""What"" Skills","- The ""Observing"" skill in the Mindfulness mod...",False
M3,Describing,Acceptance,Mindfulness,"Mindfulness ""What"" Skills","- The ""Describing"" skill in the Mindfulness mo...",False
M4,Participating,Acceptance,Mindfulness,"Mindfulness ""What"" Skills","- The ""Participating"" skill in the Mindfulness...",False
M5,Nonjudgmentally,Acceptance,Mindfulness,"Mindfulness ""How"" Skills","- The ""Nonjudgmentally"" skill in the Mindfulne...",False
M6,One-Mindfully,Acceptance,Mindfulness,"Mindfulness ""How"" Skills","- The ""One-Mindfully"" skill in the Mindfulness...",False
M7,Effectively,Acceptance,Mindfulness,"Mindfulness ""How"" Skills","- The ""Effectively"" skill in the Mindfulness m...",False
M8,Mindfulness Practice: A Spiritual Perspective,Acceptance,Mindfulness,Other Perspectives on Mindfulness,"- The ""Mindfulness Practice: A Spiritual Persp...",False
M9,Balancing Doing Mind and Being Mind,Acceptance,Mindfulness,Other Perspectives on Mindfulness,"- The ""Balancing Doing Mind and Being Mind"" sk...",False


In [210]:
DBT_SKILLS_REF.dtypes

skill_name       object
module_focus     object
module_name      object
category_name    object
skill_desc       object
use_regex          bool
dtype: object

In [209]:
VALID_SKILL_IDS = list(DBT_SKILLS_REF.index)
VALID_SKILL_IDS

['M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'IE1',
 'IE2',
 'IE3',
 'IE4',
 'IE5',
 'IE6',
 'IE7',
 'IE8',
 'IE9',
 'IE10',
 'IE11',
 'ER1',
 'ER2',
 'ER3',
 'ER4',
 'ER5',
 'ER6',
 'ER7',
 'ER8',
 'ER9',
 'ER10',
 'DT1',
 'DT2',
 'DT3',
 'DT4',
 'DT5',
 'DT6',
 'DT7',
 'DT8',
 'DT9',
 'DT10',
 'DT11',
 'DT12',
 'DT13',
 'DT14',
 'DT15',
 'DT16']

In [224]:
x = {'a','b'}
x.add('c')
x.add('c')
x

{'a', 'b', 'c'}

In [257]:
DBT_SKILLS_REF_FILEPATH = os.path.join('..', 'data', 'dbt_skills_ref.csv')
SKILL_PRESENCE_BASE_PROMPT_FILEPATH = os.path.join('..', 'data', 'skill_presence_base_prompt.txt')

DBT_SKILLS_REF=pd.read_csv(
    DBT_SKILLS_REF_FILEPATH,
    index_col='skill_id',
)
VALID_SKILL_IDS = list(DBT_SKILLS_REF.index)
with open(SKILL_PRESENCE_BASE_PROMPT_FILEPATH, 'rt') as f:
    SKILL_PRESENCE_BASE_PROMPT = f.read()

class DBTSkill:
    def __init__(self, skill_id:str):
        assert skill_id in VALID_SKILL_IDS
        self.skill_id = skill_id

        ref_row = DBT_SKILLS_REF.loc[skill_id]
        self.skill_name = ref_row.skill_name
        self.module_focus = ref_row.module_focus
        self.module_name = ref_row.module_name
        self.category_name = None
        category_name = ref_row.category_name
        if pd.notna(category_name):
            self.category_name = category_name

        self.skill_desc = ref_row.skill_desc
        self.use_regex = ref_row.use_regex
        if self.use_regex:
            regex_info = SKILL_REGEX_DICT[self.skill_id]
            self.pattern_str = regex_info['pattern_str']
            self.pattern = re.compile(self.pattern_str)
        else:
            self.skill_presence_prompt = SKILL_PRESENCE_BASE_PROMPT.replace('<SKILL_DESC>', self.skill_desc)







In [255]:
SKILL_PRESENCE_BASE_PROMPT

'Determine if a therapist discussed the following Dialectical Behavior Therapy (DBT) skill during a session:\n<SKILL_DESC>\n\nCarefully review the provided transcript between the therapist and client. Rate whether the therapist discussed the DBT skill using the following criteria:\n\nRate "1" if the therapist did any of the following:\n- Mentioned the skill by name.\n- Defined, described, explained, or taught the skill to the client.\n- Led the client through guided practice of the skill.\n- Helped apply the skill to the client\'s life.\n\nRate "0" if none of these actions took place.\n\nProvide your rating as a single number, either "1" or "0", with no further explanation.'

In [259]:
skl = DBTSkill('M1')
skl.__dict__

{'skill_id': 'M1',
 'skill_name': 'Wise Mind',
 'module_focus': 'Acceptance',
 'module_name': 'Mindfulness',
 'category_name': None,
 'skill_desc': '- The "Wise Mind" skill in the Mindfulness module of DBT refers to a state of mind where an individual integrates both rational thought (Reasonable Mind) and emotional experience (Emotion Mind) to make balanced and effective decisions.\n- It is considered a core concept in DBT, representing an inner knowledge and intuition that everyone possesses, which can guide them to a sense of truth and balanced action.\n- In practice, this skill involves recognizing when one is operating from either the Reasonable Mind or the Emotion Mind and then working towards achieving a synthesis of the two, which is the Wise Mind.\n- During a session, it might be identified by the therapist asking the client to reflect on a situation and consider what their emotions, reason, and intuition are telling them, aiming to find a middle path that incorporates all aspe